# Modelos secuenciales markovianos

Ahora estudiaremos otra modalidad de NLP, que es el modelado de secuencias categóricas. 

Hasta ahora hemos trabajado con bag-of-words, un modelo en el que se pierde la **información secuencial**, es decir, el orden en el que vienen las palabras. Estudiaremos uno de los modelos secuenciales más populares: el modelo oculto de Markov. 

## La cadena de Markov

Una cadena de Markov formaliza las probabilidades de cambio de estados en un sistema no determinístico. Por ejemplo, supongamos que nuestro sistema es el clima de nuestra localidad, y puede estar en dos estados dependiendo del día: "Lluvioso" (estado ``L``) y "Soleado" (estado ``S``). Observamos empíricamente la siguiente secuencia de estados cada día por 30 días:

``S, S, L, S, S, L, L, S, S, S, L, L, L, S, S, S, S, L, S, L, S, S, L, S, S, L, L, S, S, L``

Existen cuatro transiciones posibles: ``S`` $\rightarrow$ ``S``, ``S`` $\rightarrow$ ``L``, ``L`` $\rightarrow$ ``L`` y ``L`` $\rightarrow$ ``S``. En general, el número de transiciones en una cadena de Markov es de $M^2$ donde $M$ es el número de estados. Las cadenas de Markov cumplen con la llamada **propiedad de Markov**, que dice que, si estamos en un estado $t$, cualquier predicción sobre un estado futuro se hace con base en el estado $t$ y no en la historia completa de estados anteriores. 

Vamos a calcular una matriz de conteos con la secuencia anterior, donde contamos el número de veces que cada transición se da (se toma el índice de fila como el "antes", y el de la columna el "después"):

| | ``L`` | ``S`` |
|---|---|---|
| ``L`` | 4 | 7 |
| ``S`` | 8 | 10 |

Esta matriz la convertimos en una **matriz de transición** dividiendo cada valor el total **de su fila**

| | ``L`` | ``S`` |
|---|---|---|
| ``L`` | 0.36 | 0.64 |
| ``S`` | 0.44 | 0.56 |

De modo que **la suma de cada fila da 1**. En forma gráfica, esta cadena se representaría del siguiente modo:

<img src="resources/mc1.png">

La podemos interpretar de la siguiente manera: **si hoy es un día lluvioso, la probabilidad de que mañana llueva es del 36% y del 64% que sea soleado; asimismo, si hoy es un día soleado, la probabilidad de que mañana llueva es del 44% y del 56% de mantenerse soleado**.

## Modelos ocultos de Markov (HMM)

Continuemos con nuestro ejemplo del clima. Supongamos que, en vez de tener datos sobre si hace o no un día lluvioso, tenemos datos sobre la ropa que una persona se ha puesto cada día, la cual puede ser ``saco``, ``jacket``, ``shorts``, ``bikini``, o ``jeans``. Sabemos bien que el tipo de prenda que una persona escoge está en función del clima que hace afuera, por lo cual podríamos inferir el tipo de clima que hace en base al tipo de prendas observadas. 

Aquí entra **el modelo oculto de Markov**, que es un modelo secuencial Markoviano pero aumentado con **una capa adicional** de **estados ocultos**. Los llamamos "ocultos" porque funcionan como una **variable latente**, que a pesar de que se pueden "ver" en los datos dichos estados **no hay un etiquetado explícito** en ellos.

Volviendo a nuestro ejemplo, tenemos nuestros datos con cinco tipos de observaciones (prendas), pero también vamos a decir que tenemos **dos** estados ocultos: "lluvioso" y "soleado" (esta decisión la tomamos por observación). El modelo resultante tendría dos niveles:

1. El nivel **latente**, en donde nos podemos mover entre el estado oculto ``L`` ("lluvioso") y el estado oculto ``S`` ("soleado"), y modelamos las **probabilidades de transición** de movernos entre ``L`` y ``S``.

2. El nivel **observado**, en donde tenemos las posibles observaciones (``saco``, ``jacket``, ``shorts``, ``bikini`` y ``jeans``) con las probabilidades de que aparezcan, las cuales vienen condicionadas a los estados latentes en los que estamos y se conocen **probabilidades de emisión**. Básicamente, son de la forma "probabilidad de que mañana llueva dado que estamos en el estado invierno".  

Veámoslo gráficamente:

<img src="resources/mc2.png">

Aquí, los círculos representan estados latentes y los cuadrados observaciones. Las líneas sólidas representan probabilidades de transición y las líneas punteadas representan probabilidades de emisión. 

Adicionalmente, también hay un "nivel cero" donde tenemos la probabilidad de comienzo, es decir, la probabilidad de que el primer estado de la secuencia sea un estado latente determinado. 

### Aprendiendo el HMM con ``hmmlearn``

La biblioteca ``scikit-learn`` tenía originalmente HMMs, pero se movieron por completo al paquete ``hmmlearn``. 


In [199]:
import numpy as np
from hmmlearn import hmm
from sklearn.preprocessing import LabelEncoder

# secuencia de observaciones
x = ['jeans', 'jeans', 'jacket', 'jeans', 'shorts', 'shorts', 'shorts', 'jacket', 'jeans', 'saco', 'shorts', 'bikini', 'shorts', 'jacket', 'shorts', 'bikini', 'bikini', 'shorts', 'saco', 'bikini', 'shorts', 'shorts', 'jacket', 'jeans', 'jeans', 'saco', 'bikini', 'bikini', 'bikini', 'bikini', 'shorts', 'shorts', 'jacket', 'bikini', 'shorts', 'bikini', 'bikini', 'jacket', 'saco', 'jeans', 'jacket', 'jeans', 'bikini', 'bikini', 'bikini', 'jeans', 'saco', 'saco', 'bikini', 'bikini', 'shorts', 'jeans', 'shorts', 'bikini', 'bikini', 'shorts', 'jeans', 'bikini', 'shorts', 'bikini', 'saco', 'jeans', 'saco', 'saco', 'bikini', 'bikini', 'shorts', 'shorts', 'bikini', 'shorts', 'saco', 'bikini', 'shorts', 'bikini', 'shorts', 'saco', 'jeans', 'saco', 'jacket', 'saco', 'shorts', 'bikini', 'bikini', 'saco', 'saco', 'saco', 'bikini', 'bikini', 'shorts', 'jacket', 'shorts', 'bikini', 'bikini', 'shorts', 'jeans', 'bikini', 'bikini', 'shorts', 'saco', 'jeans', 'jeans', 'jeans', 'bikini', 'shorts', 'shorts', 'shorts', 'shorts', 'bikini', 'jeans', 'shorts', 'shorts', 'bikini', 'shorts', 'jeans', 'saco', 'jeans', 'jeans', 'saco', 'bikini', 'shorts', 'bikini', 'saco', 'jeans', 'jeans', 'bikini', 'shorts', 'shorts', 'jeans', 'bikini', 'bikini', 'bikini', 'shorts', 'jeans', 'shorts', 'shorts', 'bikini', 'jacket', 'jeans', 'jeans', 'jeans', 'shorts', 'bikini', 'shorts', 'bikini', 'shorts', 'bikini', 'jacket', 'bikini', 'bikini', 'bikini', 'bikini', 'jeans', 'jeans', 'jeans', 'jacket', 'jeans', 'shorts', 'shorts', 'shorts', 'jacket', 'jeans', 'saco', 'shorts', 'bikini', 'shorts', 'jacket', 'shorts', 'bikini', 'bikini', 'shorts', 'saco', 'bikini', 'shorts', 'shorts', 'jacket', 'jeans', 'jeans', 'saco', 'bikini', 'bikini', 'bikini', 'bikini', 'shorts', 'shorts', 'jacket', 'bikini', 'shorts', 'bikini', 'bikini', 'jacket', 'saco', 'jeans', 'jacket', 'jeans', 'bikini', 'bikini', 'bikini', 'jeans', 'saco', 'saco', 'bikini', 'bikini', 'shorts', 'jeans', 'shorts', 'bikini', 'bikini', 'shorts', 'jeans', 'bikini', 'shorts', 'bikini', 'saco', 'jeans', 'saco', 'saco', 'bikini', 'bikini', 'shorts', 'shorts', 'bikini', 'shorts', 'saco', 'bikini', 'shorts', 'bikini', 'shorts', 'saco', 'jeans', 'saco', 'jacket', 'saco', 'shorts', 'bikini', 'bikini', 'saco', 'saco', 'saco', 'bikini', 'bikini', 'shorts', 'jacket', 'shorts', 'bikini', 'bikini', 'shorts', 'jeans', 'bikini', 'bikini', 'shorts', 'saco', 'jeans', 'jeans', 'jeans', 'bikini', 'shorts', 'shorts', 'shorts', 'shorts', 'bikini', 'jeans', 'shorts', 'shorts', 'bikini', 'shorts', 'jeans', 'saco', 'jeans', 'jeans', 'saco', 'bikini', 'shorts', 'bikini', 'saco', 'jeans', 'jeans', 'bikini', 'shorts', 'shorts', 'jeans', 'bikini', 'bikini', 'bikini', 'shorts', 'jeans', 'shorts', 'shorts', 'bikini', 'jacket', 'jeans', 'jeans', 'jeans', 'shorts', 'bikini', 'shorts', 'bikini', 'shorts', 'bikini', 'jacket', 'bikini', 'bikini', 'bikini', 'bikini', 'jeans']

# LabelEncoder para codificar la secuencia en enteros {0, 1, 2, 3, 4}
enc = LabelEncoder().fit(x)

# codificamos la secuencia en un arreglo de enteros
x_enc = enc.transform(x).reshape(1, -1)

# entrenamos el modelo
model = hmm.MultinomialHMM(n_components=2, n_iter=100, verbose=True).fit(x_enc)

# veamos las probabilidades de transmisión entre estados
print(model.transmat_)

# y las probabilidades de emisión
print('Emisión estado 1: ', dict(zip(enc.classes_, model.emissionprob_[0])), '\n')
print('Emisión estado 2: ', dict(zip(enc.classes_, model.emissionprob_[1])), '\n')

[[0.27900368 0.72099632]
 [0.52475175 0.47524825]]
Emisión estado 1:  {'bikini': 0.545468634353083, 'jacket': 0.13053428448336737, 'jeans': 0.28148475527909095, 'saco': 0.02063032970877119, 'shorts': 0.021881996175687493} 

Emisión estado 2:  {'bikini': 0.1703094054087684, 'jacket': 0.029752218658917043, 'jeans': 0.12434004357809879, 'saco': 0.21286732626641336, 'shorts': 0.4627310060878023} 



         1        -524.2352             +nan
         2        -453.9804         +70.2548
         3        -453.7784          +0.2019
         4        -453.5973          +0.1812
         5        -453.4336          +0.1637
         6        -453.2843          +0.1493
         7        -453.1469          +0.1374
         8        -453.0193          +0.1276
         9        -452.9004          +0.1188
        10        -452.7899          +0.1105
        11        -452.6880          +0.1020
        12        -452.5949          +0.0931
        13        -452.5111          +0.0837
        14        -452.4369          +0.0742
        15        -452.3721          +0.0648
        16        -452.3162          +0.0559
        17        -452.2684          +0.0478
        18        -452.2277          +0.0408
        19        -452.1929          +0.0348
        20        -452.1630          +0.0299
        21        -452.1370          +0.0260
        22        -452.1141          +0.0229
        23

In [201]:
# imprimimos los estados decodificados para cada valor en la secuencia
seq_dec = model.decode(x_enc.T)[1]
print(seq_dec)

[0 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0 0 1 0 1
 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0
 1 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1 1
 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 0 1
 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 0 0 1 1 0
 0 1 0 1 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1 1 0 1
 1 0 1 0 1 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 0 1 0 0 1 1 1 1
 0 0 1 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 0
 1 0 0 1 0 1 0 1]


## Generando secuencias a partir de un HMM

Además de la predicción, una de las aplicaciones más comunes de modelos secuenciales (desde HMMs hasta redes neuronales recurrentes) es la generación de contenido. 

In [401]:
citas = [
    'Non eram nescius Brute cum quae summis ingeniis exquisitaque doctrina philosophi Graeco sermone tractavissent ea Latinis litteris mandaremus fore ut hic noster labor in varias reprehensiones incurreret nam quibusdam et iis quidem non admodum indoctis totum hoc displicet philosophari quidam autem non tam id reprehendunt si remissius agatur sed tantum studium tamque multam operam ponendam in eo non arbitrantur erunt etiam et ii quidem eruditi Graecis litteris contemnentes Latinas qui se dicant in Graecis legendis operam malle consumere postremo aliquos futuros suspicor qui me ad alias litteras vocent genus hoc scribendi etsi sit elegans personae tamen et dignitatis esse negent',
    'Contra quos omnis dicendum breviter existimo Quamquam philosophiae quidem vituperatoribus satis responsum est eo libro quo a nobis philosophia defensa et collaudata est cum esset accusata et vituperata ab Hortensio qui liber cum et tibi probatus videretur et iis quos ego posse iudicare arbitrarer plura suscepi veritus ne movere hominum studia viderer retinere non posse Qui autem si maxime hoc placeat moderatius tamen id volunt fieri difficilem quandam temperantiam postulant in eo quod semel admissum coerceri reprimique non potest ut propemodum iustioribus utamur illis qui omnino avocent a philosophia quam his qui rebus infinitis modum constituant in reque eo meliore quo maior sit mediocritatem desiderent',
    'Sive enim ad sapientiam perveniri potest non paranda nobis solum ea sed fruenda etiam sapientia est sive hoc difficile est tamen nec modus est ullus investigandi veri nisi inveneris et quaerendi defatigatio turpis est cum id quod quaeritur sit pulcherrimum etenim si delectamur cum scribimus quis est tam invidus qui ab eo nos abducat sin laboramus quis est qui alienae modum statuat industriae nam ut Terentianus Chremes non inhumanus qui novum vicinum non vult fodere aut arare aut aliquid ferre denique  non enim illum ab industria sed ab inliberali labore deterret  sic isti curiosi quos offendit noster minime nobis iniucundus labor',
    'Iis igitur est difficilius satis facere qui se Latina scripta dicunt contemnere in quibus hoc primum est in quo admirer cur in gravissimis rebus non delectet eos sermo patrius cum idem fabellas Latinas ad verbum e Graecis expressas non inviti legant quis enim tam inimicus paene nomini Romano est qui Ennii Medeam aut Antiopam Pacuvii spernat aut reiciat quod se isdem Euripidis fabulis delectari dicat Latinas litteras oderit',
    'Synephebos ego inquit potius Caecilii aut Andriam Terentii quam utramque Menandri legam',
    'A quibus tantum dissentio ut cum Sophocles vel optime scripserit Electram tamen male conversam Atilii mihi legendam putem de quo Lucilius ferreum scriptorem verum opinor scriptorem tamen ut legendus sit rudem enim esse omnino in nostris poetis aut inertissimae segnitiae est aut fastidii delicatissimi mihi quidem nulli satis eruditi videntur quibus nostra ignota sunt an Utinam ne in nemore    nihilo minus legimus quam hoc idem Graecum quae autem de bene beateque vivendo a Platone disputata sunt haec explicari non placebit Latine',
    'Quid si nos non interpretum fungimur munere sed tuemur ea quae dicta sunt ab iis quos probamus eisque nostrum iudicium et nostrum scribendi ordinem adiungimus quid habent cur Graeca anteponant iis quae et splendide dicta sint neque sint conversa de Graecis nam si dicent ab illis has res esse tractatas ne ipsos quidem Graecos est cur tam multos legant quam legendi sunt quid enim est a Chrysippo praetermissum in Stoicis legimus tamen Diogenem Antipatrum Mnesarchum Panaetium multos alios in primisque familiarem nostrum Posidonium quid Theophrastus mediocriterne delectat cum tractat locos ab Aristotele ante tractatos quid Epicurei num desistunt de isdem de quibus et ab Epicuro scriptum est et ab antiquis ad arbitrium suum scribere quodsi Graeci leguntur a Graecis isdem de rebus alia ratione compositis quid est cur nostri a nostris non legantur',
    'Quamquam si plane sic verterem Platonem aut Aristotelem ut verterunt nostri poetae fabulas male credo mererer de meis civibus si ad eorum cognitionem divina illa ingenia transferrem sed id neque feci adhuc nec mihi tamen ne faciam interdictum puto locos quidem quosdam si videbitur transferam et maxime ab iis quos modo nominavi cum inciderit ut id apte fieri possit ut ab Homero Ennius Afranius a Menandro solet Nec vero ut noster Lucilius recusabo quo minus omnes mea legant utinam esset ille Persius Scipio vero et Rutilius multo etiam magis quorum ille iudicium reformidans Tarentinis ait se et Consentinis et Siculis scribere facete is quidem sicut alia sed neque tam docti tum erant ad quorum iudicium elaboraret et sunt illius scripta leviora ut urbanitas summa appareat doctrina mediocris',
    'Ego autem quem timeam lectorem cum ad te ne Graecis quidem cedentem in philosophia audeam scribere quamquam a te ipso id quidem facio provocatus gratissimo mihi libro quem ad me de virtute misisti Sed ex eo credo quibusdam usu venire ut abhorreant a Latinis quod inciderint in inculta quaedam et horrida de malis Graecis Latine scripta deterius quibus ego assentior dum modo de isdem rebus ne Graecos quidem legendos putent res vero bonas verbis electis graviter ornateque dictas quis non legat nisi qui se plane Graecum dici velit ut a Scaevola est praetore salutatus Athenis Albucius',
    'Sed iure Mucius ego autem mirari satis non queo unde hoc sit tam insolens domesticarum rerum fastidium non est omnino hic docendi locus sed ita sentio et saepe disserui Latinam linguam non modo non inopem ut vulgo putarent sed locupletiorem etiam esse quam Graecam quando enim nobis vel dicam aut oratoribus bonis aut poetis postea quidem quam fuit quem imitarentur ullus orationis vel copiosae vel elegantis ornatus defuit Ego vero quoniam forensibus operis laboribus periculis non deseruisse mihi videor praesidium in quo a populo Romano locatus sum debeo profecto quantumcumque possum in eo quoque elaborare ut sint opera studio labore meo doctiores cives mei nec cum istis tantopere pugnare qui Graeca legere malint modo legant illa ipsa ne simulent et iis servire qui vel utrisque litteris uti velint vel si suas habent illas non magnopere desiderent',
    'Qui autem alia malunt scribi a nobis aequi esse debent quod et scripta multa sunt sic ut plura nemini e nostris et scribentur fortasse plura si vita suppetet et tamen qui diligenter haec quae de philosophia litteris mandamus legere assueverit iudicabit nulla ad legendum his esse potiora quid est enim in vita tantopere quaerendum quam cum omnia in philosophia tum id quod his libris quaeritur qui sit finis quid extremum quid ultimum quo sint omnia bene vivendi recteque faciendi consilia referenda quid sequatur natura ut summum ex rebus expetendis quid fugiat ut extremum malorum qua de re cum sit inter doctissimos summa dissensio quis alienum putet eius esse dignitatis quam mihi quisque tribuat quid in omni munere vitae optimum et verissimum sit exquirere',
    'An partus ancillae sitne in fructu habendus disseretur inter principes civitatis P Scaevolam Maniumque Manilium ab iisque M Brutus dissentiet  quod et acutum genus est et ad usus civium non inutile nosque ea scripta reliquaque eiusdem generis et legimus libenter et legemus  haec quae vitam omnem continent neglegentur nam ut sint illa vendibiliora haec uberiora certe sunt quamquam id quidem licebit iis existimare qui legerint nos autem hanc omnem quaestionem de finibus bonorum et malorum fere a nobis explicatam esse his litteris arbitramur in quibus quantum potuimus non modo quid nobis probaretur sed etiam quid a singulis philosophiae disciplinis diceretur persecuti sumus',
    'Ut autem a facillimis ordiamur prima veniat in medium Epicuri ratio quae plerisque notissima est quam a nobis sic intelleges eitam ut ab ipsis qui eam disciplinam probant non soleat accuratius explicari verum enim invenire volumus non tamquam adversarium aliquem convincere accurate autem quondam a L Torquato homine omni doctrina erudito defensa est Epicuri sententia de voluptate a meque ei responsum cum C Triarius in primis gravis et doctus adolescens ei disputationi interesset',
    'Nam cum ad me in Cumanum salutandi causa uterque venisset pauca primo inter nos de litteris quarum summum erat in utroque studium deinde Torquatus Quoniam nacti te inquit sumus aliquando otiosum certe audiam quid sit quod Epicurum nostrum non tu quidem oderis ut fere faciunt qui ab eo dissentiunt sed certe non probes eum quem ego arbitror unum vidisse verum maximisque erroribus animos hominum liberavisse et omnia tradidisse quae pertinerent ad bene beateque vivendum sed existimo te sicut nostrum Triarium minus ab eo delectari quod ista Platonis Aristoteli Theophrasti orationis ornamenta neglexerit nam illud quidem adduci vix possum ut ea quae senserit ille tibi non vera videantur',
    'Vide quantum inquam fallare Torquate oratio me istius philosophi non offendit nam et complectitur verbis quod vult et dicit plane quod intellegam et tamen ego a philosopho si afferat eloquentiam non asperner si non habeat non admodum flagitem re mihi non aeque satisfacit et quidem locis pluribus sed quot homines tot sententiae falli igitur possumus',
    'Quam ob rem tandem inquit non satisfacit te enim iudicem aequum puto modo quae dicat ille bene noris',
    'Nisi mihi Phaedrum inquam tu mentitum aut Zenonem putas quorum utrumque audivi cum mihi nihil sane praeter sedulitatem probarent omnes mihi Epicuri sententiae satis notae sunt atque eos quos nominavi cum Attico nostro frequenter audivi cum miraretur ille quidem utrumque Phaedrum autem etiam amaret cotidieque inter nos ea quae audiebamus conferebamus neque erat umquam controversia quid ego intellegerem sed quid probarem',
    'Quid igitur est inquit audire enim cupio quid non probes Principio inquam in physicis quibus maxime gloriatur primum totus est alienus Democritea dicit perpauca mutans sed ita ut ea quae corrigere vult mihi quidem depravare videatur ille atomos quas appellat id est corpora individua propter soliditatem censet in infinito inani in quo nihil nec summum nec infimum nec medium nec ultimum nec extremum sit ita ferri ut concursionibus inter se cohaerescant ex quo efficiantur ea quae sint quaeque cernantur omnia eumque motum atomorum nullo a principio sed ex aeterno tempore intellegi convenire',
    'Epicurus autem in quibus sequitur Democritum non fere labitur quamquam utriusque cum multa non probo tum illud in primis quod cum in rerum natura duo quaerenda sint unum quae materia sit ex qua quaeque res efficiatur alterum quae vis sit quae quidque efficiat de materia disseruerunt vim et causam efficiendi reliquerunt sed hoc commune vitium illae Epicuri propriae ruinae censet enim eadem illa individua et solida corpora ferri deorsum suo pondere ad lineam hunc naturalem esse omnium corporum motum',
    'Deinde ibidem homo acutus cum illud ocurreret si omnia deorsus e regione ferrentur et ut dixi ad lineam numquam fore ut atomus altera alteram posset attingere itaque  attulit rem commenticiam declinare dixit atomum perpaulum quo nihil posset fieri minus ita effici complexiones et copulationes et adhaesiones atomorum inter se ex quo efficeretur mundus omnesque partes mundi quaeque in eo essent Quae cum tota res est ficta pueriliter tum ne efficit quidem quod vult nam et ipsa declinatio ad libidinem fingitur  ait enim declinare atomum sine causa quo nihil turpius physico quam fieri quicquam sine causa dicere  et illum motum naturalem omnium ponderum ut ipse constituit e regione inferiorem locum petentium sine causa eripuit atomis nec tamen id cuius causa haec finxerat assecutus est',
    'Nam si omnes atomi declinabunt nullae umquam cohaerescent sive aliae declinabunt aliae suo nutu recte ferentur primum erit hoc quasi provincias atomis dare quae recte quae oblique ferantur deinde eadem illa atomorum in quo etiam Democritus haeret turbulenta concursio hunc mundi ornatum efficere non poterit ne illud quidem physici credere aliquid esse minimum quod profecto numquam putavisset si a Polyaeno familiari suo geometrica discere maluisset quam illum etiam ipsum dedocere Sol Democrito magnus videtur quippe homini erudito in geometriaque perfecto huic pedalis fortasse tantum enim esse censet quantus videtur vel paulo aut maiorem aut minorem',
    'Ita quae mutat ea corrumpit quae sequitur sunt tota Democriti atomi inane imagines quae eidola nominant quorum incursione non solum videamus sed etiam cogitemus infinitio ipsa quam apeirian vocant tota ab illo est tum innumerabiles mundi qui et oriantur et intereant cotidie Quae etsi mihi nullo modo probantur tamen Democritum laudatum a ceteris ab hoc qui eum unum secutus esset nollem vituperatum',
]

citas = [ cita.lower().split() for cita in citas ]

# codificamos
enc = LabelEncoder().fit([y for l in citas for y in l])
citas_enc = np.concatenate([enc.transform(y).reshape(-1, 1) for y in citas])
citas_len = [len(y) for y in citas]

citas_model = hmm.MultinomialHMM(n_iter=200, n_components=2, verbose=True).fit(citas_enc, citas_len)

Fitting a model with 1977 free scalar parameters with only 1949 data points will result in a degenerate solution.


[100, 105, 101, 68, 12, 81, 133, 127, 96, 136, 123, 102, 73, 107, 55, 18, 61, 92, 79, 123, 95, 62]


         1      -13783.9204             +nan
         2      -12300.5570       +1483.3634
         3      -12300.4160          +0.1410
         4      -12300.1606          +0.2554
         5      -12299.6963          +0.4643
         6      -12298.9278          +0.7685
         7      -12297.8355          +1.0923
         8      -12296.5124          +1.3231
         9      -12295.0498          +1.4626
        10      -12293.4758          +1.5740
        11      -12291.8029          +1.6730
        12      -12289.9865          +1.8164
        13      -12287.7783          +2.2082
        14      -12284.5596          +3.2187
        15      -12279.0962          +5.4634
        16      -12269.0296         +10.0666
        17      -12250.0000         +19.0296
        18      -12214.9991         +35.0009
        19      -12157.0988         +57.9004
        20      -12079.8303         +77.2684
        21      -12003.2560         +76.5744
        22      -11944.0518         +59.2042
        23

In [414]:
print(' '.join(enc.inverse_transform(citas_model.sample(3)[0])))
print(' '.join(enc.inverse_transform(citas_model.sample(3)[0])))
print(' '.join(enc.inverse_transform(citas_model.sample(5)[0])))
print(' '.join(enc.inverse_transform(citas_model.sample(7)[0])))
print(' '.join(enc.inverse_transform(citas_model.sample(10)[0])))
print(' '.join(enc.inverse_transform(citas_model.sample(15)[0])))


intellegerem solum inimicus
eum omnes efficere
sitne arbitrantur quae multam latinas
potiora sedulitatem probes quam erat summa cognitionem
modo enim fere atomis rerum ut ut cum legendi sit
eum efficiantur deorsum corporum in existimare potest cur concursio num a quae graecis iudicium duo
